# Chapter 20. Concurrent Execution

This chapter focuses on `concurrent.futures.Executor` classes that encapuslate the pattern of "spawning a bunch of independent threads and collecting the results in a queue"

Here the author introduces the concept of "futures"--objects representing the asynchronous execution of an operation, similar to JS promises.

Executors are the most important high-level feature while futures are low-level objects.

## Concurrent Web Downloads

Concurrency is essential for efficient network I/O: instead of idly waiting for remote machines, the application should do sth else until a response comes back.

Three simples programs to download images of 20 country flags from the web
 - `flags.py`: runs sequentially
 - `flags_threadpool.py` uses the `concurrent.futures` pacakge
 - `flags_asyncio.py` uses the `asyncio`

### A Sequential Download Script

In [2]:
!pip install httpx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00


In [ ]:
import time
from pathlib import Path
from typing import Callable

import httpx

POP20_CC = ('CN IN US ID BR PK NG BD RU KR '
            'MX PH VN ET EG DE IR TR CD FR').split()

BASE_URL = 'https://www.fluentpython.com/data/flags'
DEST_DIR = Path('downloaded')

# save the img bytes to filename in the DEST_DIR
def save_flag(img: bytes, filenames: str) -> None:
  (DEST_DIR / filename).write_bytes(img)

def get_flag(cc: str) -> bytes:
  url = f'{BASE_URL}/{cc}/{cc}.gif'.lower()
  resp = httpx.get(url, timeout=6.1, # it's a good practice to add a sensible timeout
                   follow_redirects=True) # By default, HTTPX does not follow redirects
  resp.raise_for_status()
  return resp.content

# key function to compare with the concurrent implementations
def download_many(cc_list: list[str]) -> int:
  for cc in sorted(cc_list):
    image = get_flag(cc)
    save_flag(image, f'{cc}.gif')
    print(cc, end=" ", flush=True) # display one country code at a time in the same line
                                   # flush=True is needed because by default Python output is line buffered
  return len(cc_list)

def main(downloader: Callable[[list[str]], int]) -> None:
  DEST_DIR.mkdir(exist_ok=True)
  t0 = time.perf_counter()
  count = downloader(POP20_CC)
  elapsed = time.perf_counter() - t0
  print(f"\n{count} downloads in {elapsed:.2f}s")

if __name__ == "__main__":
  main(download_many)


In [4]:
!python flags.py

BD BR CD CN DE EG ET FR ID IN IR KR MX NG PH PK RU TR US VN 
20 downloads in 6.43s


### Downloading with concurrent.futures

Main Features: `ThreadPoolExecutor` and `ProcessPoolExectuor` classes, which implement an API to submit cllables for execution in different threads or processes. The classes transparently manage a pool of worker threads or processes, and queues to distribute jobs and collect results.

In [ ]:
# flags_threadpool.py
from concurrent import futures

from flags import save_flag, get_flag, main

def download_one(cc: str):
  image = get_flag(cc)
  save_flag(image, f'{cc}.gif')
  print(cc, end=" ", flush=True)
  return cc

def download_many(cc_list: list[str]) -> int:
  # instantiate the thread pool executor as a context manager
  # exectuor.__exit__ will call executor.shutdown(wait=True) which will block until all threads are done
  with futures.ThreadPoolExecutor() as executor:
    res = executor.map(download_one, sorted(cc_list))

  return len(list(res))

if __name__ == "__main__":
  main(download_many)



In [8]:
!python flags_threadpool.py

BD CN DE BR EG CD ET ID FR KR IN IR MX NG PH RU PK TR US VN 
20 downloads in 1.29s


In [9]:
import os
os.cpu_count() + 4

6